In [2]:
import numpy as np
import gurobipy as gp
import pandas as pd

In [2]:
# Parts 1-3

In [16]:
roiMod=gp.Model()
roiModX = roiMod.addMVar(10)

In [29]:
              #print, TV,  SEO, AdWords, Facebook, LinkedIn, Instagram, Snapchat, Twitter, Email
obj1 = np.array([3.1,  4.9, 2.4, 3.9,     1.6,      2.4,      4.6,       2.6,      3.3,     4.4])
#obj1 = pd.read_csv("ROI_data.csv")

roiMod.setObjective(gp.quicksum(obj1[j] * roiModX[j] for j in range(10)),sense=gp.GRB.MAXIMIZE)

conlist=[0]*4
conlist[0] = roiMod.addConstr(roiModX[0] + roiModX[1] - roiModX[4] - roiModX[9] <= 0) #print and TV <= facebook and email
conlist[1] = roiMod.addConstr(roiModX[4:8] - 2 * roiModX[2:3] >= 0) #high social media spending
conlist[2] = roiMod.addConstrs(roiModX[j] <= 3 for j in range(10)) #no more than $3M
conlist[3] = roiMod.addConstr(sum(roiModX[:]) <= 10) #budget of $10M

In [30]:
roiMod.Params.OutputFlag = 0  #tell gurobi to shut up!!

roiMod.optimize() # solve the model

In [31]:
roiMod.objVal # print the optimal objective value

45.6

In [32]:
roiModX.x # print the optimal decision variables

array([0., 3., 0., 1., 0., 0., 3., 0., 0., 3.])

In [8]:
# Parts 4-5

In [23]:
              #print, TV,  SEO, AdWords, Facebook, LinkedIn, Instagram, Snapchat, Twitter, Email
obj2 = np.array([4.9,  2.3, 2.4, 3.9,     4.4,      4.6,      2.6,       1.9,      3.7,     2.6])
roiMod.setObjective(gp.quicksum(obj2[j] * roiModX[j] for j in range(10)),sense=gp.GRB.MAXIMIZE)

In [24]:
roiMod.Params.OutputFlag = 0  #tell gurobi to shut up!!

roiMod.optimize() # solve the model

In [25]:
roiMod.objVal # print the optimal objective value

45.6

In [26]:
roiModX.x # print the optimal decision variables

array([3., 0., 0., 1., 3., 3., 0., 0., 0., 0.])

In [13]:
#Yes, the allocations are different between the two, but both produce the same overall ROI.

In [14]:
#Assuming the first ROI data is correct, if you were to use the second allocation,
#how much lower would the objective be relative to the optimal objective?

In [10]:
# optimize the second model before running
delta = roiModX.x * [obj2 - obj1]
sum(sum(delta))

20.4

In [16]:
#Assuming the second ROI data is correct, if you used the first allocation,
#how much lower would the objective be relative to the optimal objective?

In [15]:
# optimize the first model before running
delta = roiModX.x * [obj1 - obj2]
sum(sum(delta))

19.200000000000003

In [18]:
# Question 6

In [33]:
roiModX.SAObjLow

array([-inf,  3.9, -inf,  3.3, -inf, -inf,  3.9, -inf, -inf,  3.9])

In [34]:
roiModX.SAObjUp

array([3.9, inf, 3.9, 4.4, 3.9, 3.9, inf, 3.9, 3.9, inf])

In [21]:
# Question 7

In [24]:
ROI = [0]
old_budget = [10]
new_budget = []

for i in range(12):
    
             #print, TV,  SEO, AdWords, Facebook, LinkedIn, Instagram, Snapchat, Twitter, Email
    obj3 = np.array([3.1,  4.9, 2.4, 3.9,     1.6,      2.4,      4.6,       2.6,      3.3,     4.4])
    #obj1 = pd.read_csv("ROI_data.csv")

    roiMod.setObjective(gp.quicksum(obj3[j] * roiModX[j] / 100 for j in range(10)),sense=gp.GRB.MAXIMIZE)
    new_budget += gp.quicksum(obj3[j] * roiModX[j] / 200 for j in range(10))
    old_budget += new_budget[-1]

    conlist=[0]*4
    conlist[0] = roiMod.addConstr(roiModX[0] + roiModX[1] - roiModX[4] - roiModX[9] <= 0) #print and TV <= facebook and email
    conlist[1] = roiMod.addConstr(roiModX[4:8] - 2 * roiModX[2:3] >= 0) #high social media spending
    conlist[2] = roiMod.addConstrs(roiModX[j] <= 3 for j in range(10)) #no more than $3M
    conlist[3] = roiMod.addConstr(sum(roiModX[:]) <= new_budget) #budget of $10M

    roiMod.Params.OutputFlag = 0  #tell gurobi to shut up!!
    roiMod.optimize() # solve the model
    
    ROI += [roiMod.objVal/1000]
    print(roiModX.x)

print(ROI)
print(new_budget)
print(old_budget)

TypeError: unsupported operand type(s) for +: 'MLinExpr' and 'list'